In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models,Sequential,constraints
from tensorflow.keras.layers import Conv1D,MaxPooling1D,Dropout,Dense,Flatten,Layer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,hamming_loss
from sklearn.utils import class_weight
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from matplotlib import pyplot,colors
from scipy.stats import pearsonr

In [ ]:
spectra=pd.read_csv('C:\Users\ks\Desktop\Spectra Identification\Spectra_Checked_baseline.csv',header=0) # Read spectra data
X=spectra.iloc[:,1:].values
labels=pd.read_csv('Labels_Checked_baseline.csv',header=0) # Read labels data
Y=labels.iloc[:,1:].values
X=X_data.transpose()
Y=Y_data.transpose()
X_train, X_test, y_train, y_test= train_test_split(X, Y, test_size=0.3, random_state=42)
X_train=np.expand_dims(X_train,axis=2) # Expand to include channel axis for CNN
X_test=np.expand_dims(X_test,axis=2)# Expand to include channel axis for CNN


In [ ]:
# Definition of custom Keras layer for Lorentzians
class Lorentz_layer(Layer):
    def __init__(self, units=5,kernel_constraint=None,
               bias_constraint=None,**kwargs):
        super(Lorentz_layer,self).__init__(**kwargs)
        self.units = units
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        
    def build(self,input_shape):
        self.w = tf.Variable(name="kernel",   initial_value=90*tf.eye(input_shape[-1], num_columns=self.units,
                 dtype='float32'),constraint=self.kernel_constraint,trainable=True)
        
        b_init = tf.keras.initializers.RandomUniform(minval=0, maxval=100, seed=None)
        self.b = tf.Variable(name="bias",initial_value=b_init(shape=(self.units,), dtype='float32'),
                             constraint=self.bias_constraint,
                             trainable=True)
        self.x=(np.linspace(0,3600,901,dtype='float32')).reshape(901,1)
    
    def call(self,inputs):
        x0=tf.matmul(inputs,self.w)+K.epsilon()
        gamma=self.b+K.epsilon()
        val=(1/(np.pi*gamma))*(1/(1+(tf.subtract(self.x,tf.expand_dims(x0,axis=1))/gamma)**2))
        return val


In [ ]:
# Definition of custom layer for summation of Cauchy distributions
class sum_layer(Layer):
    def __init__(self, units=1,kernel_constraint=None,
               bias_constraint=None,**kwargs):
        super(sum_layer,self).__init__(**kwargs)
        self.units = units
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
    def build(self,input_shape):
        w_init = tf.keras.initializers.RandomUniform(minval=0.01, maxval=1, seed=None)
        self.w = tf.Variable(name="kernel",   initial_value=w_init(shape=(input_shape[-1], self.units),
                 dtype='float32'),constraint=self.kernel_constraint,trainable=True)
        
        self.b = None
    def call(self,inputs):
        val=tf.matmul(inputs,self.w)
        val=tf.divide(tf.subtract(val, tf.reduce_min(val)), tf.subtract(tf.reduce_max(val), tf.reduce_min(val))+K.epsilon())
        return val

In [ ]:
# Build Keras model structure
my_layer=Lorentz_layer(units=15,kernel_constraint=constraints.NonNeg(),bias_constraint=constraints.NonNeg())
final_layer=sum_layer(kernel_constraint=constraints.NonNeg())
Input_layer=keras.Input(shape=(901,1))
conv1=Conv1D(filters=10, kernel_size=5,activation='relu',padding='same')
pool1=MaxPooling1D()
pool2=MaxPooling1D()
conv2=Conv1D(filters=10, kernel_size=5,activation='relu',padding='same')
drop=Dropout(0.5)
flat=Flatten()
Dense1=Dense(20,activation='relu')
Dense2=Dense(20,activation='relu')
Dense3=Dense(15,activation='relu',name='X0')
Sig=Dense(14,activation='sigmoid',name='Labels')
x=conv1(Input_layer)
x=pool1(x)
x=conv2(x)
x=pool2(x)
x=drop(x)
x=flat(x)

x=Dense1(x)
x=Dense3(x)
op_label=Sig(x)
y=Dense2(x)
spec=my_layer(y)
op_spec=final_layer(spec)

model=keras.Model(Input_layer,[op_label,op_spec])
model.summary()



In [ ]:
# Weighting loss function for less represented classes
def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', classes=[0.,1.], y=y_true[:, i])
    return weights
weights=calculating_class_weights(y_train)
def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        y_true=tf.cast(y_true,tf.float32)
        y_pred=tf.cast(y_pred,tf.float32)
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss

In [ ]:
def KLDerror(y_true,y_pred):
    x=K.softmax(y_true,axis=1)
    y=K.softmax(y_pred,axis=1)
    kl=keras.losses.KLDivergence()
    return kl(x,y)

In [ ]:

opt = keras.optimizers.Adam(learning_rate=1E-3) #Set learning rate and optimizer
model.compile(loss=[get_weighted_loss(weights),KLDerror],optimizer=opt,loss_weights=[1,500]) # Realtive importance of each loss
history=model.fit(X_train,[y_train,X_train],epochs=3000,validation_data=(X_test, [y_test,X_test]),verbose=1)

In [ ]:
model.save('C:\Users\ks\Desktop\Spectra Identification\SavedModels\NewArch_Water')